In [7]:
from utils import setup_bmc5_env
import os

setup_bmc5_env()

os.environ['CUDA_CORE'] = "cuda:1" 

In [8]:
from model_setup import setup_model_tokenizer

tokenizer, model = setup_model_tokenizer()

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.57s/it]


In [10]:
del model, tokenizer

NameError: name 'model' is not defined

In [11]:
import torch

torch.cuda.empty_cache()


In [ ]:
gc.collect()

In [2]:
from sentence_transformers import SentenceTransformer, util
import torch

device_to_load_sentence_transformer = torch.device('cuda:1')
torch.cuda.set_device(device_to_load_sentence_transformer)

model_s = SentenceTransformer('all-MiniLM-L6-v2')

model_s

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [3]:
# Assuming model_s is your SentenceTransformer model
device = next(model_s.parameters()).device
print(f"Model is on device: {device}")


Model is on device: cuda:1


In [4]:
print(f"Device Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")


Device Name: NVIDIA RTX A6000


In [5]:
import torch

def print_gpu_memory_stats():
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            total_memory = torch.cuda.get_device_properties(i).total_memory
            allocated_memory = torch.cuda.memory_allocated(i)
            cached_memory = torch.cuda.memory_reserved(i)
            free_memory = total_memory - (allocated_memory + cached_memory)

            print(f"GPU {i}:")
            print(f"  Total Memory: {total_memory / (1024**3):.2f} GB")
            print(f"  Allocated Memory: {allocated_memory / (1024**3):.2f} GB")
            print(f"  Cached Memory: {cached_memory / (1024**3):.2f} GB")
            print(f"  Free Memory: {free_memory / (1024**3):.2f} GB\n")
    else:
        print("No CUDA devices are available")

print_gpu_memory_stats()


GPU 0:
  Total Memory: 44.55 GB
  Allocated Memory: 0.00 GB
  Cached Memory: 0.00 GB
  Free Memory: 44.55 GB

GPU 1:
  Total Memory: 44.55 GB
  Allocated Memory: 0.08 GB
  Cached Memory: 0.10 GB
  Free Memory: 44.37 GB

GPU 2:
  Total Memory: 44.55 GB
  Allocated Memory: 0.00 GB
  Cached Memory: 0.00 GB
  Free Memory: 44.55 GB

GPU 3:
  Total Memory: 44.55 GB
  Allocated Memory: 0.00 GB
  Cached Memory: 0.00 GB
  Free Memory: 44.55 GB



In [1]:
from utils import find_gpu_with_memory_threshold

# Example usage: Find a GPU with at least 2 GB of available memory
gpu_name = find_gpu_with_memory_threshold(0.4)
print(f"GPU Name with at least 2 GB free: {gpu_name}")


free_memory_gb=44.5538330078125, threshold_gb=0.4
free_memory_gb=44.5538330078125, threshold_gb=0.4
free_memory_gb=44.5511474609375, threshold_gb=0.4
free_memory_gb=44.5538330078125, threshold_gb=0.4
GPU Name with at least 2 GB free: None


In [3]:
def find_gpu_with_memory_threshold(threshold_gb):
    """
    Finds the first GPU with available memory greater than the specified threshold and returns its name.
    This version uses nvidia-smi to get accurate free memory information.
    """
    import torch
    import subprocess
    import re

    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            try:
                # Running nvidia-smi command to get memory usage
                smi_output = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader', '-i', str(i)], encoding='utf-8')
                free_memory_gb = float(smi_output.strip()) / 1024  # Convert MiB to GiB

                if free_memory_gb > threshold_gb:
                    gpu_name = torch.cuda.get_device_name(i)
                    print(f"Num {i}, GPU: {gpu_name}, Free Memory: {free_memory_gb} GB")
                    # return gpu_name
            except Exception as e:
                print(f"Error querying GPU {i} memory using nvidia-smi: {e}")

    print("No GPU found with the required free memory threshold.")
    return None


gpu_name = find_gpu_with_memory_threshold(0.4)
print(f"GPU Name with at least 2 GB free: {gpu_name}")


GPU: NVIDIA RTX A6000, Free Memory: 44.2109375 GB
GPU: NVIDIA RTX A6000, Free Memory: 16.404296875 GB
GPU: NVIDIA RTX A6000, Free Memory: 15.166015625 GB
No GPU found with the required free memory threshold.
GPU Name with at least 2 GB free: None
